In [1]:
from python_readiness import *


In [ ]:
req_txt = "..."

In [2]:
reqs = sorted(set(Requirement(r).name for r in parse_requirements_txt(req_txt)))

In [27]:
import datetime as dt
from dataclasses import dataclass
from collections import Counter


@dataclass(frozen=True)
class Result:
    package: str
    version: Version
    support: PythonSupport
    date: dt.datetime


async def get_results(packages: list[str], python_version: tuple[int, int]):
    session = CachedSession()

    supports: list[tuple[Version | None, PythonSupport, dict[str, Any] | None]]
    supports = await asyncio.gather(
        *(
            dist_support(session, p, python_version, monotonic_support=False, exclude_newer=None)
            for p in packages
        )
    )
    package_support = dict(zip(packages, supports, strict=True))

    ret = []
    counter = Counter()

    for package, (version, support, file_proof) in sorted(
        package_support.items(), key=lambda x: (-x[1][1].value, x[0])
    ):
        assert (file_proof is None) == (support < PythonSupport.EXPLICIT_INDICATION)
        if file_proof is not None:
            assert version is not None
            upload_time = dt.datetime.fromisoformat(file_proof["upload-time"])
            ret.append(Result(package, version, support, upload_time))
        counter[support] += 1

    await session.close()

    assert sum(counter.values()) == len(packages)
    print(f"\nout of {len(packages)} packages:")
    for k, v in sorted(counter.items(), key=lambda x: x[0].value, reverse=True):
        print(k.name, v)

    return ret

In [ ]:
py310_results = await get_results(reqs, (3, 10))
py311_results = await get_results(reqs, (3, 11))
py312_results = await get_results(reqs, (3, 12))
py313_results = await get_results(reqs, (3, 13))

In [ ]:
py310_final = dt.datetime(2021, 10, 4, tzinfo=dt.timezone.utc)
py311_final = dt.datetime(2022, 10, 24, tzinfo=dt.timezone.utc)
py312_final = dt.datetime(2023, 10, 2, tzinfo=dt.timezone.utc)
py313_final = dt.datetime(2024, 10, 7, tzinfo=dt.timezone.utc)

In [32]:
import matplotlib.pyplot as plt

In [ ]:
py310_dates_relative = sorted([(r.date - py310_final).days for r in py310_results])
py311_dates_relative = sorted([(r.date - py311_final).days for r in py311_results])
py312_dates_relative = sorted([(r.date - py312_final).days for r in py312_results])
py313_dates_relative = sorted([(r.date - py313_final).days for r in py313_results])

plt.figure(figsize=(8, 5))
plt.plot(py310_dates_relative, range(1, len(py310_dates_relative) + 1), label="3.10")
plt.plot(py311_dates_relative, range(1, len(py311_dates_relative) + 1), label="3.11")
plt.plot(py312_dates_relative, range(1, len(py312_dates_relative) + 1), label="3.12")
plt.plot(py313_dates_relative, range(1, len(py313_dates_relative) + 1), label="3.13")

plt.title('Python adoption')
plt.xlabel('Days since final release')
plt.ylabel('Packages definitely supporting Python')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
wheel_support = {PythonSupport.has_classifier_and_explicit_wheel, PythonSupport.has_explicit_wheel}

py310_dates_relative = sorted([(r.date - py310_final).days for r in py310_results if r.support in wheel_support])
py311_dates_relative = sorted([(r.date - py311_final).days for r in py311_results if r.support in wheel_support])
py312_dates_relative = sorted([(r.date - py312_final).days for r in py312_results if r.support in wheel_support])
py313_dates_relative = sorted([(r.date - py313_final).days for r in py313_results if r.support in wheel_support])

plt.figure(figsize=(8, 5))
plt.plot(py310_dates_relative, range(1, len(py310_dates_relative) + 1), label="3.10")
plt.plot(py311_dates_relative, range(1, len(py311_dates_relative) + 1), label="3.11")
plt.plot(py312_dates_relative, range(1, len(py312_dates_relative) + 1), label="3.12")
plt.plot(py313_dates_relative, range(1, len(py313_dates_relative) + 1), label="3.13")

plt.title('Python adoption amongst packages with version specific wheels')
plt.xlabel('Days since final release')
plt.ylabel('Packages definitely supporting Python')
plt.legend()
plt.grid(True)
plt.show()

In [22]:
def plot_python_adoption(
    dates: list[dt.datetime],
    vlines: list[tuple[dt.datetime, str]],
    python_version: tuple[int, int],
):
    adoption_dates = sorted(dates)
    adoption_counts = range(1, len(dates) + 1)

    plt.figure(figsize=(8, 5))
    plt.plot(adoption_dates, adoption_counts)

    for date, label in vlines:
        plt.axvline(x=date, color='r', linestyle='--', label=label)

    python_str = ".".join(map(str, python_version))
    plt.title(f'Python {python_str} Adoption')
    plt.xlabel('Date')
    plt.ylabel(f'Packages definitely supporting Python {python_str}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_python_adoption(
    [r.date for r in py311_results],
    # https://peps.python.org/pep-0664/
    [
        (dt.datetime(2022, 5, 8), "3.11b1"),
        (dt.datetime(2022, 8, 8), "3.11rc1"),
        (dt.datetime(2022, 10, 24), "3.11"),
    ],
    (3, 11),
)

In [ ]:
plot_python_adoption(
    [r.date for r in py312_results],
    # https://peps.python.org/pep-0693/
    [
        (dt.datetime(2023, 5, 22), "3.12b1"),
        (dt.datetime(2023, 8, 6), "3.12rc1"),
        (dt.datetime(2023, 10, 2), "3.12"),
    ],
    (3, 12),
)

In [ ]:
plot_python_adoption(
    [r.date for r in py313_results],
    # https://peps.python.org/pep-0719/
    [
        (dt.datetime(2024, 5, 8), "3.13b1"),
        (dt.datetime(2024, 8, 1), "3.13rc1"),
        (dt.datetime(2024, 10, 7), "3.13"),
    ],
    (3, 13),
)